In [17]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import os 
os.getcwd()

'/home/selezov_su/notebooks/Promics/Panoram'

In [14]:
import requests

def call_string(request_url, params):
    return requests.post(request_url, data=params)

def main(
        prot_list,
        string_api_url="https://version-11-5.string-db.org/api",
        method='network?',
        output_format = "tsv",
        network_type="functional",
        # add_nodes=1000,
    ):

    request_url = "/".join([string_api_url, output_format, method])

    params = {
        "identifiers" : "%0d".join(prot_list),     # your protein
        "species" : 9606,                          # species NCBI identifier 
        "caller_identity" : "www.awesome_app.org", # your app name
        "required_score" : "500",                  # threshold of significance
    }

    resp =  call_string(request_url, params).text
    
    tab = pd.DataFrame([line.split("\t") for line in resp.split("\n")])
    tab.columns = tab.iloc[0, :]
    tab = tab.iloc[1:, :]
    
    return tab

In [18]:
with open("Data/whole_protein_set.txt", "r") as file:
    proteins = file.read().splitlines() 
    

from UniProtMapper import ProtMapper

mapper = ProtMapper()

result, failed = mapper.get(
    ids=proteins, from_db="UniProtKB_AC-ID", to_db="Gene_Name"
)
    
    
resp = main(result["To"].to_list()).iloc[:-1]
resp

Setting fields to `None` to retrieve all available fields...


Fetched: 98 / 98


,stringId_A,stringId_B,preferredName_A,preferredName_B,ncbiTaxonId,score,nscore,fscore,pscore,ascore,escore,dscore,tscore
1,9606.ENSP00000170150,9606.ENSP00000306361,BPIFB2,FGA,9606,0.652,0,0,0,0.434,0,0,0.41
2,9606.ENSP00000170150,9606.ENSP00000306361,BPIFB2,FGA,9606,0.652,0,0,0,0.434,0,0,0.41
3,9606.ENSP00000170150,9606.ENSP00000306099,BPIFB2,FGB,9606,0.673,0,0,0,0.3,0,0,0.552
4,9606.ENSP00000170150,9606.ENSP00000306099,BPIFB2,FGB,9606,0.673,0,0,0,0.3,0,0,0.552
5,9606.ENSP00000170150,9606.ENSP00000253354,BPIFB2,BPIFB1,9606,0.831,0,0,0,0,0,0.3,0.768
...,...,...,...,...,...,...,...,...,...,...,...,...,...
722,9606.ENSP00000432660,9606.ENSP00000458162,CFL1,ACTG1,9606,0.991,0,0,0,0.279,0.798,0.8,0.752
723,9606.ENSP00000445175,9606.ENSP00000458162,LDHA,ACTG1,9606,0.556,0,0,0,0.107,0,0,0.523
724,9606.ENSP00000445175,9606.ENSP00000458162,LDHA,ACTG1,9606,0.556,0,0,0,0.107,0,0,0.523
725,9606.ENSP00000446007,9606.ENSP00000458162,VIM,ACTG1,9606,0.683,0,0,0,0.086,0.349,0,0.511


In [19]:
import networkx as nx
from pyvis.network import Network


# Create directed graph with networkx
G = nx.DiGraph()
G = nx.from_pandas_edgelist(
    resp.loc[resp.escore.astype(float) > 0.1], 
    source="preferredName_A", 
    target="preferredName_B", 
    edge_attr=["score", "nscore", "fscore"],
)

# Plot with pyvis
net = Network(
    cdn_resources='remote',
    notebook=True,
    directed = True,
    select_menu = True, # Show part 1 in the plot (optional)
    filter_menu = True, # Show part 2 in the plot (optional)
)
net.show_buttons() # Show part 3 in the plot (optional)
net.from_nx(G) # Create directly from nx graph
net.show('test.html')

test.html
